In [69]:
from astropy.table import Table, join
import numpy as np

In [3]:
freqs = [98, 223.5 ,339.7672758867]
data = Table.read('/home/jotter/nrao/summer_research_2018/tables/r0.5_catalog_bgfit_mar21_ulim.fits')
cac = Table.read('/home/jotter/nrao/summer_research_2018/tables/r0.5_mar21_calc_vals.fits')

In [4]:
def specind_compute(freq1, freq2, flux1, flux2, flux1_err, flux2_err):
    freqdiff = np.log10(freq2) - np.log10(freq1)
    specind = (np.log10(flux2)-np.log10(flux1))/freqdiff
    error = np.sqrt((flux1_err/(freqdiff*np.log(10)*flux1))**2 + (flux2_err/(freqdiff*np.log(10)*flux2))**2)
    return specind, error

In [5]:
b3flux = data['ap_flux_B3'].data
b3flux_err = data['ap_flux_err_B3'].data
b6flux = data['ap_flux_B6'].data
b6flux_err = data['ap_flux_err_B6'].data
b7flux = data['ap_flux_B7'].data
b7flux_err = data['ap_flux_err_B7'].data

In [6]:
b6b7_specind, b6b7_specind_err = specind_compute(freqs[2],freqs[1],b7flux, b6flux, b7flux_err, b6flux_err)
b3b6_specind, b3b6_specind_err = specind_compute(freqs[1],freqs[0],b6flux, b3flux, b6flux_err, b3flux_err)

In [7]:
b3b6_ind = np.where(np.isnan(b3b6_specind) == False)[0]
b6b7_ind = np.where(np.isnan(b6b7_specind) == False)[0]
print(len(b3b6_ind), len(b6b7_ind))

42 34


In [8]:
sys_err = 0.20 #20% systematic error

b3flux = data['ap_flux_B3'].data
b3flux_err = np.sqrt(data['ap_flux_err_B3'].data**2 + (sys_err*b3flux)**2)
b6flux = data['ap_flux_B6'].data
b6flux_err = np.sqrt(data['ap_flux_err_B6'].data**2 + (sys_err*b6flux)**2)
b7flux = data['ap_flux_B7'].data
b7flux_err = np.sqrt(data['ap_flux_err_B7'].data**2 + (sys_err*b7flux)**2)

In [9]:
b6b7_specind, b6b7_specind_err_sys = specind_compute(freqs[2],freqs[1],b7flux, b6flux, b7flux_err, b6flux_err)
b3b6_specind, b3b6_specind_err_sys = specind_compute(freqs[1],freqs[0],b6flux, b3flux, b6flux_err, b3flux_err)

In [10]:
print(b3b6_specind[b3b6_ind], b3b6_specind_err_sys[b3b6_ind])
print(b6b7_specind[b6b7_ind], b6b7_specind_err_sys[b6b7_ind])

[ 2.06951976  4.04406965  2.22915097  1.96195886  1.79241549  2.0311904
 -1.9276605   2.33421134  1.91836748  3.35025701  2.3530512   1.56727741
  1.92306402  2.45733916  2.17632605  1.99955867  2.39251537  1.98617753
  1.86660575 -1.38129033  1.91635857  1.99726657  1.91596848  2.40227483
  1.86006657  0.80799336  2.75304329  1.88634822  1.67444959  3.24592345
  2.03704059  1.78980895  1.69137715  1.4395901   2.29630769  0.90048557
  1.68549343  1.99692554  1.12566122  1.57329538  1.76940851  2.09141107] [0.34310249 0.36164027 0.34309744 0.34395758 0.34872215 0.34308182
 1.54831179 0.34309522 0.3462319  0.3432438  0.34325239 0.34613968
 0.34574301 0.34421104 0.34803586 0.34308647 0.34314899 0.34336369
 0.36087017 0.61182138 0.34325431 0.34315677 0.36573069 0.34316828
 0.35166059 0.34321035 0.34391949 0.34392411 0.34351874 0.36721177
 0.34340533 0.34315227 0.34334138 0.39877803 0.37086647 0.3761203
 0.48940227 0.34535324 0.42286077 0.36297917 0.4716159  0.38075765]
[ 0.7379854   1.3404

In [11]:
##now find where the vaues agree with 2
b3b6_specind2 = np.where(np.abs(b3b6_specind - 2) <=  b3b6_specind_err_sys)[0]
b3b6_specind2_nosys = np.where(np.abs(b3b6_specind - 2) <=  b3b6_specind_err)[0]

print(len(b3b6_specind2), len(b3b6_specind2_nosys))

b6b7_specind2 = np.where(np.abs(b6b7_specind - 2) <=  b6b7_specind_err_sys)[0]
b6b7_specind2_nosys = np.where(np.abs(b6b7_specind - 2) <=  b6b7_specind_err)[0]

print(len(b6b7_specind2), len(b6b7_specind2_nosys))

26 8
22 3


In [14]:
np.intersect1d(b3b6_specind2, b6b7_specind2)

array([10, 12, 16, 25, 26, 28, 31, 32, 33, 35, 44, 66])

In [15]:
np.intersect1d(b3b6_specind2_nosys, b6b7_specind2_nosys)

array([], dtype=int64)

In [29]:
both_agree = np.where(np.abs(b6b7_specind - b3b6_specind) <  np.sqrt(b6b7_specind_err_sys**2 + b3b6_specind_err_sys**2))[0]

In [30]:
both_agree

array([10, 12, 16, 20, 21, 24, 25, 26, 27, 28, 31, 32, 33, 34, 35, 39, 40,
       44, 65, 66, 67, 68])

In [44]:
##now find where the vaues less than 2
b3b6_specindl2 = np.where(2 - b3b6_specind >  b3b6_specind_err_sys)[0]
b3b6_specindl2_nosys = np.where(2 - b3b6_specind >  b3b6_specind_err)[0]

print(len(b3b6_specindl2), len(b3b6_specindl2_nosys))

b6b7_specindl2 = np.where(2 - b6b7_specind >  b6b7_specind_err_sys)[0]
b6b7_specindl2_nosys = np.where(2 - b6b7_specind >  b6b7_specind_err)[0]

print(len(b6b7_specindl2), len(b6b7_specindl2_nosys))
print(np.intersect1d(b3b6_specindl2, b6b7_specindl2))

8 19
7 17
[39]


In [46]:
##now find where the vaues greater than 2
b3b6_specindg2 = np.where(b3b6_specind - 2>  b3b6_specind_err_sys)[0]
b3b6_specindg2_nosys = np.where(b3b6_specind - 2>  b3b6_specind_err)[0]

print(len(b3b6_specindg2), len(b3b6_specindg2_nosys))

b6b7_specindg2 = np.where(b6b7_specind - 2>  b6b7_specind_err_sys)[0]
b6b7_specindg2_nosys = np.where(b6b7_specind - 2 >  b6b7_specind_err)[0]

print(b3b6_specindg2, b6b7_specindg2)
print(np.intersect1d(b3b6_specindg2, b6b7_specindg2))

8 15
[ 8 18 20 24 27 34 40 45] [29 30 62 64 69]
[]


In [40]:
avg_alpha = (b3b6_specind[both_agree] + b6b7_specind[both_agree])/2
avg_alpha_err = np.sqrt(b3b6_specind_err_sys[both_agree]**2 + b6b7_specind_err_sys[both_agree]**2)/2

print(avg_alpha)
print(avg_alpha_err)

[1.65121235 1.93661611 2.16158885 2.24516902 1.6714802  2.33731267
 1.81721607 2.02538138 2.37841654 2.16007733 1.77466747 1.81781577
 1.69016363 2.21394659 1.83922126 1.02239433 2.44764829 1.56726057
 1.35431181 2.28717825 0.85473547 1.73791936]
[0.37908573 0.37872181 0.37981582 0.37876014 0.37943875 0.37897753
 0.38013198 0.37872094 0.37873606 0.37878803 0.37876652 0.37873851
 0.38497825 0.37874168 0.38082369 0.37875931 0.37893609 0.37888595
 0.44591258 0.3793037  1.08348315 0.3986705 ]


In [37]:
both_agree[np.where(np.abs(avg_alpha - 2) < avg_alpha_err)[0]]

array([10, 12, 16, 20, 21, 24, 25, 26, 27, 28, 31, 32, 33, 34, 35, 66, 68])

In [47]:
both_agree[np.where(avg_alpha - 2 > avg_alpha_err)[0]] #avg greater than 2

array([40])

In [48]:
both_agree[np.where(2 - avg_alpha > avg_alpha_err)[0]] #avg greater than 2

array([39, 44, 65, 67])

In [60]:
only_b3b6_ind = np.setdiff1d(b3b6_ind, b6b7_ind)

In [61]:
only_b3b6_ind

array([ 6,  8, 11, 41, 48, 49, 50, 70])

In [64]:
print(b3b6_specind[only_b3b6_ind])
print(b3b6_specind_err_sys[only_b3b6_ind])

[2.06951976 4.04406965 1.79241549 1.88634822 2.03704059 1.78980895
 1.69137715 2.09141107]
[0.34310249 0.36164027 0.34872215 0.34392411 0.34340533 0.34315227
 0.34334138 0.38075765]


In [65]:
b3b6_specind[only_b3b6_ind] + b3b6_specind_err_sys[only_b3b6_ind]

array([2.41262225, 4.40570992, 2.14113765, 2.23027233, 2.38044592,
       2.13296121, 2.03471853, 2.47216873])

In [66]:
only_b3b6_ind

array([ 6,  8, 11, 41, 48, 49, 50, 70])

In [72]:
fb = Table.read('/home/jotter/nrao/summer_research_2018/tables/Forbrich2016_r0.5_mar21.fits')
fb.rename_column('Seq_1','Seq')

In [73]:
specind_tab = Table((b3b6_ind, b3b6_specind[b3b6_ind], b3b6_specind_err_sys[b3b6_ind],
                     b6b7_specind[b3b6_ind], b6b7_specind_err_sys[b3b6_ind]),
                    names=('Seq', 'b3b6_alpha', 'b3b6_alpha_err', 'b6b7_alpha', 'b6b7_alpha_err'))

In [74]:
match = join(specind_tab, fb, keys='Seq')

Seq,b3b6_alpha,b3b6_alpha_err,b6b7_alpha,b6b7_alpha_err,fwhm_maj_B3,fwhm_maj_err_B3,fwhm_min_B3,fwhm_min_err_B3,pa_B3,pa_err_B3,gauss_amp_B3,gauss_amp_err_B3,RA_B3,RA_err_B3,DEC_B3,DEC_err_B3,ap_flux_B3,ap_flux_err_B3,fwhm_maj_deconv_B3,fwhm_maj_deconv_err_B3,fwhm_min_deconv_B3,fwhm_min_deconv_err_B3,pa_deconv_B3,pa_deconv_err_B3,SNR_B3,RMS_B3,ar_deconv_B3,D_ID,fwhm_maj_B6,fwhm_maj_err_B6,fwhm_min_B6,fwhm_min_err_B6,pa_B6,pa_err_B6,gauss_amp_B6,gauss_amp_err_B6,RA_B6,RA_err_B6,DEC_B6,DEC_err_B6,success_B6,ap_flux_B6,ap_flux_err_B6,fwhm_maj_deconv_B6,fwhm_maj_deconv_err_B6,fwhm_min_deconv_B6,fwhm_min_deconv_err_B6,pa_deconv_B6,pa_deconv_err_B6,SNR_B6,ar_deconv_B6,fwhm_maj_B7,fwhm_maj_err_B7,fwhm_min_B7,fwhm_min_err_B7,pa_B7,pa_err_B7,gauss_amp_B7,gauss_amp_err_B7,RA_B7,RA_err_B7,DEC_B7,DEC_err_B7,success_B7,ap_flux_B7,ap_flux_err_B7,fwhm_maj_deconv_B7,fwhm_maj_deconv_err_B7,fwhm_min_deconv_B7,fwhm_min_deconv_err_B7,pa_deconv_B7,pa_deconv_err_B7,SNR_B7,ar_deconv_B7,upper_lim_B6,upper_lim_B7,upper_lim_B3,Seq_2,Rad,RAJ2000,e_RAJ2000,DEJ2000,e_DEJ2000,Spk,e_Spk,alpha,e_alpha,Ref,COUP,Ltc,VISION,Separation
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,AU,AU,AU,,arcmin,deg,s,deg,arcsec,mJy,mJy,,,,,[10-7W],,arcsec
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes8,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes8,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int16,float32,float64,float64,float64,float64,float32,float32,float32,float32,bytes10,int16,float32,bytes16,float64
11,1.7924154923992122,0.34872215346960056,nan,nan,0.12508210993658392,0.010340954500372333,0.07738150510535492,0.0063974644881915135,63.87105340817135,0.1171777427736119,0.00042892873791004993,3.5459341681562276e-05,83.81413482476985,1.1444981184015256e-06,-5.377732925184,8.646495562344217e-07,0.0005267397030781578,2.6459894084720006e-05,0.08435570004661445,0.010340954500372333,0.009750403626335292,0.0063974644881915135,1.3197653459129894,0.1171777427736119,8.222765350482419,5.1860705e-05,8.651508519993413,13,0.15964335921635386,0.0,0.06117036435877132,0.0,250.2004256854907,0.0,0.00048677623736149755,0.0,83.81413767212715,0.0,-5.377732005185563,0.0,True,0.0023087293491329645,2.6718869645249473e-05,nan,nan,nan,nan,nan,nan,4.699289971199373,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,N/A,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,10.654308473754133,nan,nan,233,0.28,83.8141426666667,7.310000000000002e-05,-5.37778622222222,0.0008300000000000001,0.4975,0.0025,-0.68,0.21,Z,0,nan,05351539-0522399,0.19391704900403609
14,-1.9276605001826117,1.5483117870624536,2.1737429893865134,3.385190971512436,0.09634824107607774,0.003102183002897431,0.06788046217540548,0.0021856768092064073,46.17639666589673,0.06370633447393365,0.00046002072376798874,1.4811674723502008e-05,83.81042393726936,3.187079583501615e-07,-5.377417792980893,3.143399465396786e-07,0.00038203434196243456,1.3921741916596627e-05,nan,nan,nan,nan,nan,nan,20.359742224942206,1.9848407e-05,nan,16,0.027765674516556003,0.0,0.027765674516556003,0.0,132.81065859813407,0.0,0.0001475595810916275,0.0,83.81042216459109,0.0,-5.3774171697576865,0.0,True,7.796519398504607e-05,9.700683584763233e-05,nan,nan,nan,nan,nan,nan,1.386662141194327,nan,0.021532390823584432,0.0,0.018212091643364403,0.0,-68.3155872938164,0.0,0.0004162559406232181,0.0,83.81042301230663,0.0,-5.3774162555227045,0.0,True,0.0001937819641795176,0.00011981376562372211,nan,nan,nan,nan,nan,nan,2.81267995275496,nan,22.13425342520799,8.188860349944783,10.637880314826772,184,0.14,8